In [47]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import os
import time


In [49]:
web = 'https://www.nba.com/stats/players/advanced'
driver = webdriver.Chrome("C:/Users/Techa/Desktop/DataSet/webscraping/chromedriver_win32/chromedriver.exe")
driver.get(web)
time.sleep(3)
#acctept_btn= driver.find_element_by_xpath(""" //*[@id="onetrust-accept-btn-handler"]""").click()
#time.sleep(1)

select = Select(driver.find_element_by_xpath(""" //*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select"""))
select.select_by_visible_text("All")
time.sleep(2)

In [50]:
# now that we have the page that contains the required data
# we rae going to set the page source for our BeautifulSoup
src = driver.page_source
soup = BeautifulSoup(src, 'lxml')

table = soup.find('table', class_='Crom_table__p1iZz')

In [51]:
headers = table.find_all('th')
headerlist = [h.text.strip() for h in headers[1:]]

In [52]:
headerlist

['PLAYER',
 'TEAM',
 'AGE',
 'GP',
 'W',
 'L',
 'MIN',
 'OFFRTG',
 'DEFRTG',
 'NETRTG',
 'AST%',
 'AST/TO',
 'AST\xa0RATIO',
 'OREB%',
 'DREB%',
 'REB%',
 'TO\xa0RATIO',
 'EFG%',
 'TS%',
 'USG%',
 'PACE',
 'PIE',
 'POSS',
 'FGM',
 'FGA',
 'FGM\xa0PG',
 'FGA\xa0PG',
 'FG%',
 'GP RANK',
 'W RANK',
 'L RANK',
 'MIN RANK',
 'OFFRTG RANK',
 'DEFRTG RANK',
 'NETRTG RANK',
 'AST% RANK',
 'AST/TO RANK',
 'AST\xa0Ratio RANK',
 'OREB% RANK',
 'DREB% RANK',
 'REB% RANK',
 'TO\xa0Ratio RANK',
 'eFG% RANK',
 'TS% RANK',
 'USG% RANK',
 'PACE RANK',
 'PIE RANK',
 'FGM RANK',
 'FGA RANK',
 'FGM\xa0PG RANK',
 'FGA\xa0PG RANK',
 'FG% RANK']

In [53]:
#we are going to be adjusting the headerlist abit, as we don't want all the header with 'RANK in the name'
headerlist= [a for a in headerlist if not 'RANK' in a ]
headerlist

['PLAYER',
 'TEAM',
 'AGE',
 'GP',
 'W',
 'L',
 'MIN',
 'OFFRTG',
 'DEFRTG',
 'NETRTG',
 'AST%',
 'AST/TO',
 'AST\xa0RATIO',
 'OREB%',
 'DREB%',
 'REB%',
 'TO\xa0RATIO',
 'EFG%',
 'TS%',
 'USG%',
 'PACE',
 'PIE',
 'POSS',
 'FGM',
 'FGA',
 'FGM\xa0PG',
 'FGA\xa0PG',
 'FG%']

If you noticed, our headlist still contain some hidden column which are not visible in the site. This might want to introduce some errors. But we are going ot deal with this shortly.

In [54]:
rows= table.find_all('tr')[1:]

In [55]:
p1 = rows[0]

In [56]:
#lets take a fine look at the structure of the data for the first player.
print(p1.prettify())

<tr>
 <td class="Crom_text__NpR1_ Crom_stickyRank__aN66p">
  1
 </td>
 <td class="Crom_text__NpR1_ Crom_primary__EajZu Crom_stickySecondColumn__29Dwf">
  <a class="Anchor_anchor__cSc3P" data-has-children="true" data-has-more="false" data-is-external="false" href="/stats/player/201143">
   Al Horford
  </a>
 </td>
 <td class="Crom_text__NpR1_">
  <a class="Anchor_anchor__cSc3P" data-has-children="true" data-has-more="false" data-is-external="false" href="/stats/team/1610612738/traditional">
   BOS
  </a>
 </td>
 <td>
  36
 </td>
 <td>
  19
 </td>
 <td>
  11
 </td>
 <td>
  8
 </td>
 <td>
  30.7
 </td>
 <td>
  114.9
 </td>
 <td>
  108.8
 </td>
 <td>
  6.1
 </td>
 <td>
  12.8
 </td>
 <td>
  5.18
 </td>
 <td>
  29.4
 </td>
 <td>
  5.1
 </td>
 <td>
  18.0
 </td>
 <td>
  11.7
 </td>
 <td>
  5.7
 </td>
 <td>
  49.2
 </td>
 <td>
  49.7
 </td>
 <td>
  10.4
 </td>
 <td>
  96.83
 </td>
 <td>
  8.6
 </td>
 <td>
  1174
 </td>
</tr>



In [57]:
len(rows)

216

In [58]:
player_stats = [[ td.getText().strip() for td in rows[i].find_all('td')[1:] ]for i in range(len(rows))]

In [59]:
player_stats[1]

['Derrick White',
 'BOS',
 '28',
 '19',
 '11',
 '8',
 '29.3',
 '114.7',
 '113.3',
 '1.4',
 '10.5',
 '2.56',
 '17.1',
 '2.1',
 '7.5',
 '4.9',
 '6.7',
 '65.1',
 '67.7',
 '15.7',
 '97.02',
 '9.4',
 '1123']

In [60]:
#lets create a dataframe for our stats. 
#But be informed, this will most likely run into an error. Since there are some hidden col, 
#the headlist col won't match the col of data extracted

#-----------------------------START-------------
#stats = pd.DataFrame(player_stats, columns=headerlist)
#stats
#----------------------------END-----------------

#if you run the code beteen the start and end above, it will run into a value error.
# to avoid interrupting the program, I am going to catch the exception with try and except
try:
    stats = pd.DataFrame(player_stats, columns=headerlist)
    stats
except ValueError:
    print("28 columns passed, passed data had 23 columns, So fix this")
    

28 columns passed, passed data had 23 columns, So fix this


In [61]:
#Yup, as expected, we have 5 more columns in our header list that do not have a data.
#thankfully that are the last 5 columns in the header list 
# we are going to drop them before creating the dataframe
#Another way to avoid this kind of error is to extract each column separated in a for loop.

headerlist = headerlist[:-5]
stats= pd.DataFrame(player_stats, columns=headerlist)
stats.head()

,PLAYER,TEAM,AGE,GP,W,L,MIN,OFFRTG,DEFRTG,NETRTG,...,OREB%,DREB%,REB%,TO RATIO,EFG%,TS%,USG%,PACE,PIE,POSS
0,Al Horford,BOS,36,19,11,8,30.7,114.9,108.8,6.1,...,5.1,18.0,11.7,5.7,49.2,49.7,10.4,96.83,8.6,1174
1,Derrick White,BOS,28,19,11,8,29.3,114.7,113.3,1.4,...,2.1,7.5,4.9,6.7,65.1,67.7,15.7,97.02,9.4,1123
2,Jaylen Brown,BOS,26,19,11,8,37.3,114.3,111.4,2.9,...,3.7,10.7,7.3,11.8,56.3,58.1,26.7,96.46,10.5,1431
3,Jayson Tatum,BOS,25,19,11,8,39.9,116.9,111.3,5.6,...,3.3,22.5,13.3,8.9,52.7,58.8,29.4,96.07,16.6,1519
4,Marcus Smart,BOS,29,19,11,8,33.9,115.0,110.3,4.6,...,3.4,8.7,6.1,11.3,55.6,59.8,19.4,97.28,10.3,1304
